<a href="https://colab.research.google.com/github/SMSinclair/DS-Unit-4-Sprint-3-Deep-Learning/blob/master/LS_DS_431_RNN_and_LSTM_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 1*

# Recurrent Neural Networks and Long Short Term Memory (LSTM)

![Monkey at a typewriter](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Chimpanzee_seated_at_typewriter.jpg/603px-Chimpanzee_seated_at_typewriter.jpg)

It is said that [infinite monkeys typing for an infinite amount of time](https://en.wikipedia.org/wiki/Infinite_monkey_theorem) will eventually type, among other things, the complete works of Wiliam Shakespeare. Let's see if we can get there a bit faster, with the power of Recurrent Neural Networks and LSTM.

This text file contains the complete works of Shakespeare: https://www.gutenberg.org/files/100/100-0.txt

Use it as training data for an RNN - you can keep it simple and train character level, and that is suggested as an initial approach.

Then, use that trained RNN to generate Shakespearean-ish text. Your goal - a function that can take, as an argument, the size of text (e.g. number of characters or lines) to generate, and returns generated text of that size.

Note - Shakespeare wrote an awful lot. It's OK, especially initially, to sample/use smaller data and parameters, so you can have a tighter feedback loop when you're trying to get things running. Then, once you've got a proof of concept - start pushing it more!

In [0]:
# TODO - Words, words, mere words, no matter from the heart.
import newspaper

In [0]:
ap = newspaper.build('http://www.gutenberg.org/files/100/100-h/100-h.htm')
len(ap.articles)

0

In [0]:
article_text = ''

for article in ap.articles:
  try:
    article.download()
    article.parse()
    article_text += '\n\n' + article.text
  except:
    print('Failed: ' + article.url)
  
article_text = article_text.split('\n\n')[1]
print(article_text)

IndexError: list index out of range

In [0]:
import numpy
import urllib.request as u  # the lib that handles the url stuff

In [0]:
url = 'http://www.gutenberg.org/files/100/100-0.txt'
data = u.urlopen(url)
txt = data.read()
txt = str(txt)

In [0]:
splits = ['THE SONNETS', 'WELL THAT ENDS WELL', 'THE TRAGEDY OF ANTONY AND CLEOPATRA',
          'AS YOU LIKE IT', 'THE COMEDY OF ERRORS', 'THE TRAGEDY OF CORIOLANUS', 'CYMBELINE',
          'THE TRAGEDY OF HAMLET', 'PRINCE OF DENMARK', 'THE FIRST PART OF KING HENRY THE FOURTH',
          'THE SECOND PART OF KING HENRY THE FOURTH', 'THE LIFE OF KING HENRY',
          'THE FIRST PART OF HENRY THE SIXTH', 'THE SECOND PART OF KING HENRY THE SIXTH',
          'THE THIRD PART OF KING HENRY THE SIXTH', 'KING HENRY THE EIGHTH', 'KING JOHN',
          'THE TRAGEDY OF JULIUS CAESAR', 'THE TRAGEDY OF KING LEAR', 'LABOUR\\xe2\\x80\\x99S LOST',
         'MACBETH', 'MEASURE FOR MEASURE', 'THE MERCHANT OF VENICE',
          'THE MERRY WIVES OF WINDSOR', 'A MIDSUMMER NIGHT','MUCH ADO ABOUT NOTHING',
          'OTHELLO, THE MOOR OF VENICE','MOOR OF VENICE', 'PERICLES, PRINCE OF TYRE', 'KING RICHARD THE SECOND',
          'KING RICHARD THE THIRD', 'THE TRAGEDY OF ROMEO AND JULIET', 'THE TAMING OF THE SHREW',
          'THE TEMPEST', 'THE LIFE OF TIMON OF ATHENS', 'THE TRAGEDY OF TITUS ANDRONICUS',
          'THE HISTORY OF TROILUS AND CRESSIDA', 'TWELFTH NIGHT',
          'THE TWO GENTLEMEN OF VERONA', 'THE TWO NOBLE KINSMEN', 'THE WINTER',
          'A LOVER', 'THE PASSIONATE PILGRIM', 'THE PHOENIX AND THE TURTLE',
          'THE RAPE OF LUCRECE', 'VENUS AND ADONIS']
txt = txt[txt.find('VENUS AND ADONIS')+len('VENUS AND ADONIS'):]
first_part = ""
indices = []
for search in splits:
    index = txt.find("%s" % search)
    indices.append(index)
    continue
    if index != -1:
        first_part = txt[:index]
        break

parts = [txt[indices[i]:indices[i+1]] for i in range(len(indices)-1)]

In [0]:
chars = list(set(parts[8])) # split and remove duplicate characters. convert to list.

num_chars = len(chars) # the number of unique characters
txt_data_size = len(article_text)

print("unique characters : ", num_chars)
print("txt_data_size : ", txt_data_size)

In [0]:
# one hot encode
char_to_int = dict((c, i) for i, c in enumerate(chars)) # "enumerate" retruns index and value. Convert it to dictionary
int_to_char = dict((i, c) for i, c in enumerate(chars))
print(char_to_int)
print("----------------------------------------------------")
print(int_to_char)
print("----------------------------------------------------")
# integer encode input data
integer_encoded = [char_to_int[i] for i in article_text] # "integer_encoded" is a list which has a sequence converted from an original data to integers.
print(integer_encoded)
print("----------------------------------------------------")
print("data length : ", len(integer_encoded))

In [0]:
# hyperparameters
class RNN():
    
    def __init__(self):
        
        self.iteration = 1000
        self.sequence_length = 40
        self.batch_size = round((txt_data_size /sequence_length)+0.5) # = math.ceil
        self.hidden_size = 500  # size of hidden layer of neurons.  
        self.learning_rate = 1e-1


        # model parameters

        self.W_xh = np.random.randn(hidden_size, num_chars)*0.01     # weight input -> hidden. 
        self.W_hh = np.random.randn(hidden_size, hidden_size)*0.01   # weight hidden -> hidden
        self.W_hy = np.random.randn(num_chars, hidden_size)*0.01     # weight hidden -> output

        self.b_h = np.zeros((hidden_size, 1)) # hidden bias
        self.b_y = np.zeros((num_chars, 1)) # output bias

        self.h_prev = np.zeros((hidden_size,1)) # h_(t-1)
        
    def forwardprop(self, inputs, targets, h_prev):
        
        # Since the RNN receives the sequence, the weights are not updated during one sequence.
        xs, hs, ys, ps = {}, {}, {}, {} # dictionary
        hs[-1] = np.copy(self.h_prev) # Copy previous hidden state vector to -1 key value.
        loss = 0 # loss initialization
    
        for t in range(len(inputs)): # t is a "time step" and is used as a key(dic).  
        
            xs[t] = np.zeros((num_chars,1)) 
            xs[t][inputs[t]] = 1
            hs[t] = np.tanh(np.dot(self.W_xh, xs[t]) + np.dot(self.W_hh, hs[t-1]) + self.b_h) # hidden state. 
            ys[t] = np.dot(self.W_hy, hs[t]) + self.b_y # unnormalized log probabilities for next chars
            ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars. 
        
            # Softmax. -> The sum of probabilities is 1 even without the exp() function, but all of the elements are positive through the exp() function.
            loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss). Efficient and simple code

 #         y_class = np.zeros((num_chars, 1)) 
  #         y_class[targets[t]] =1
  #         loss += np.sum(y_class*(-np.log(ps[t]))) # softmax (cross-entropy loss)        

        return loss, ps, hs, xs

    def backprop(self, ps, inputs, hs, xs, targets):
        # d = derivative
        # Wapital W is weight matrix
        # b = bias
        # y = output
        # h = hidden state
        # x = input
        # {}s saved information like x saved

        dWxh, dWhh, dWhy = np.zeros_like(self.W_xh), np.zeros_like(self.W_hh), np.zeros_like(self.W_hy) # make all zero matrices.
        dbh, dby = np.zeros_like(self.b_h), np.zeros_like(self.b_y)
        dhnext = np.zeros_like(hs[0]) # (hidden_size,1) 

        # reversed
        for t in reversed(range(len(inputs))):
            dy = np.copy(ps[t]) # shape (num_chars,1).  "dy" means "dloss/dy"
            dy[targets[t]] -= 1 # backprop into y. After taking the soft max in the input vector, subtract 1 from the value of the element corresponding to the correct label.
            dWhy += np.dot(dy, hs[t].T)
            dby += dy 
            dh = np.dot(self.W_hy.T, dy) + dhnext # backprop into h. 
            dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity #tanh'(x) = 1-tanh^2(x)
            dbh += dhraw
            dWxh += np.dot(dhraw, xs[t].T)
            dWhh += np.dot(dhraw, hs[t-1].T)
            dhnext = np.dot(self.W_hh.T, dhraw)
        for dparam in [dWxh, dWhh, dWhy, dbh, dby]: 
            np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients.  
    
        return dWxh, dWhh, dWhy, dbh, dby
    
    def train(self, article_text, char_to_int):
        data_pointer = 0
        
        # memory variables for Adagrad
        mWxh, mWhh, mWhy = np.zeros_like(self.W_xh), np.zeros_like(self.W_hh), np.zeros_like(self.W_hy)
        mbh, mby = np.zeros_like(self.b_h), np.zeros_like(self.b_y) 

        for i in range(iteration):
            h_prev = np.zeros((self.hidden_size,1)) # reset RNN memory
            data_pointer = 0 # go from start of data
    
            for b in range(self.batch_size):
        
                inputs = [char_to_int[ch] for ch in article_text[data_pointer:data_pointer+sequence_length]]
                targets = [char_to_int[ch] for ch in article_text[data_pointer+1:data_pointer+sequence_length+1]] # t+1        
            
                if (data_pointer+self.sequence_length+1 >= len(article_text) and b == batch_size-1): # processing of the last part of the input data. 
#             targets.append(char_to_int[txt_data[0]])   # When the data doesn't fit, add the first char to the back.
                    targets.append(char_to_int[" "])   # When the data doesn't fit, add space(" ") to the back.


                # forward
                loss, ps, hs, xs = self.forwardprop(inputs, targets, h_prev)
#         print(loss)
    
                # backward
                dWxh, dWhh, dWhy, dbh, dby = self.backprop(ps, inputs, hs, xs, targets) 
        
        
    # perform parameter update with Adagrad
                for param, dparam, mem in zip([self.W_xh, self.W_hh, self.W_hy, self.b_h, self.b_y], 
                                    [dWxh, dWhh, dWhy, dbh, dby], 
                                    [mWxh, mWhh, mWhy, mbh, mby]):
                    mem += dparam * dparam # elementwise
                    param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update      
    
                data_pointer += sequence_length # move data pointer
        
        if i % 100 == 0:
            print ('iter %d, loss: %f' % (i, loss)) # print progress
            
    def predict(self, test_char, length, num_chars):
        x = np.zeros((num_chars, 1)) 
        x[char_to_int[test_char]] = 1
        ixes = []
        h = np.zeros((hidden_size,1))

        for t in range(length):
            h = np.tanh(np.dot(self.W_xh, x) + np.dot(self.W_hh, h) + self.b_h) 
            y = np.dot(self.W_hy, h) + self.b_y
            p = np.exp(y) / np.sum(np.exp(y)) 
            ix = np.random.choice(range(num_chars), p=p.ravel()) # ravel -> rank0
            # "ix" is a list of indexes selected according to the soft max probability.
            x = np.zeros((num_chars, 1)) # init
            x[ix] = 1 
            ixes.append(ix) # list
        txt = test_char + ''.join(int_to_char[i] for i in ixes)
        print ('----\n %s \n----' % (txt, ))
        
    def encode(self)

## Retry in Tensorflow

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
tf.enable_eager_execution()

import numpy as np
import os
import time

In [2]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 
                                       'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1122304/1115394 [==============================] - 0s 0us/step


In [3]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [4]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [5]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

65 unique characters


In [0]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [7]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '$' :   3,
  '&' :   4,
  "'" :   5,
  ',' :   6,
  '-' :   7,
  '.' :   8,
  '3' :   9,
  ':' :  10,
  ';' :  11,
  '?' :  12,
  'A' :  13,
  'B' :  14,
  'C' :  15,
  'D' :  16,
  'E' :  17,
  'F' :  18,
  'G' :  19,
  ...
}


In [8]:
# Show how the first 13 characters from the text are mapped to integers
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'First Citizen' ---- characters mapped to int ---- > [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [9]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//seq_length

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

F
i
r
s
t


In [10]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [0]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [12]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target data: 'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [13]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 18 ('F')
  expected output: 47 ('i')
Step    1
  input: 47 ('i')
  expected output: 56 ('r')
Step    2
  input: 56 ('r')
  expected output: 57 ('s')
Step    3
  input: 57 ('s')
  expected output: 58 ('t')
Step    4
  input: 58 ('t')
  expected output: 1 (' ')


In [14]:
# Batch size
BATCH_SIZE = 64
steps_per_epoch = examples_per_epoch//BATCH_SIZE

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<DatasetV1Adapter shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [0]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [0]:
if tf.test.is_gpu_available():
  rnn = tf.keras.layers.CuDNNGRU
else:
  import functools
  rnn = functools.partial(
    tf.keras.layers.GRU, recurrent_activation='sigmoid')

In [0]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    rnn(rnn_units,
        return_sequences=True,
        recurrent_initializer='glorot_uniform',
        stateful=True),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [0]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [42]:
# check shape of the output
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [43]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (64, None, 256)           16640     
_________________________________________________________________
cu_dnngru_3 (CuDNNGRU)       (64, None, 1024)          3938304   
_________________________________________________________________
dense_3 (Dense)              (64, None, 65)            66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


In [0]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [46]:
sampled_indices

array([18, 45, 45,  3, 21, 37,  4, 25, 64, 63, 63, 14, 11,  5, 12, 52, 29,
       58, 52, 49, 27,  8,  1,  3, 35, 33, 50, 17,  8, 50, 46, 64,  1,  1,
        3,  8, 20, 50, 53, 27,  4,  4, 17, 58, 48, 47, 16, 20, 61, 11, 60,
        0, 42, 16, 32, 20, 54, 47, 13, 34, 21, 55, 15,  8,  4, 36, 51, 13,
       32, 42, 44, 21, 24, 39,  7, 22, 58, 44, 46, 30, 27, 57, 15, 53, 26,
       39, 32, 64, 39, 34,  4,  2, 26, 35, 16, 48,  5, 35, 64, 60])

In [47]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 "\nThey're here with me already, whispering, rounding\n'Sicilia is a so-forth:' 'tis far gone,\nWhen I s"

Next Char Predictions: 
 "Fgg$IY&MzyyB;'?nQtnkO. $WUlE.lhz  $.HloO&&EtjiDHw;v\ndDTHpiAVIqC.&XmATdfILa-JtfhROsCoNaTzaV&!NWDj'Wzv"


In [48]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 65)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.173641


In [0]:
model.compile(
    optimizer = tf.train.AdamOptimizer(),
    loss = loss)

In [0]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [51]:
EPOCHS=30

history = model.fit(dataset.repeat(), 
                    epochs=EPOCHS, 
                    steps_per_epoch=steps_per_epoch, 
                    callbacks=[checkpoint_callback])

Epoch 1/30


W0819 23:15:59.982353 140147329111936 util.py:244] Unresolved object in checkpoint: (root).optimizer
W0819 23:15:59.983670 140147329111936 util.py:244] Unresolved object in checkpoint: (root).optimizer.optimizer
W0819 23:15:59.984383 140147329111936 util.py:244] Unresolved object in checkpoint: (root).optimizer.global_step
W0819 23:15:59.989206 140147329111936 util.py:244] Unresolved object in checkpoint: (root).optimizer.optimizer.beta1_power
W0819 23:15:59.995097 140147329111936 util.py:244] Unresolved object in checkpoint: (root).optimizer.optimizer.beta2_power
W0819 23:15:59.995967 140147329111936 util.py:244] Unresolved object in checkpoint: (root).optimizer.optimizer's state 'm' for (root).layer_with_weights-0.embeddings
W0819 23:15:59.997226 140147329111936 util.py:244] Unresolved object in checkpoint: (root).optimizer.optimizer's state 'm' for (root).layer_with_weights-1.kernel
W0819 23:15:59.998144 140147329111936 util.py:244] Unresolved object in checkpoint: (root).optimizer.

174/174 [==============================] - 13s 74ms/step - loss: 2.7508
Epoch 2/30
174/174 [==============================] - 11s 64ms/step - loss: 1.9658
Epoch 3/30
174/174 [==============================] - 11s 64ms/step - loss: 1.6978
Epoch 4/30
174/174 [==============================] - 11s 65ms/step - loss: 1.5478
Epoch 5/30
174/174 [==============================] - 11s 65ms/step - loss: 1.4574
Epoch 6/30
174/174 [==============================] - 12s 67ms/step - loss: 1.3983
Epoch 7/30
174/174 [==============================] - 12s 67ms/step - loss: 1.3529
Epoch 8/30
174/174 [==============================] - 12s 68ms/step - loss: 1.3125
Epoch 9/30
174/174 [==============================] - 12s 69ms/step - loss: 1.2807
Epoch 10/30
174/174 [==============================] - 12s 69ms/step - loss: 1.2470
Epoch 11/30
174/174 [==============================] - 12s 68ms/step - loss: 1.2180
Epoch 12/30
174/174 [==============================] - 12s 68ms/step - loss: 1.1863
Epoch 13/30


In [52]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_30'

In [0]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [54]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (1, None, 256)            16640     
_________________________________________________________________
cu_dnngru_4 (CuDNNGRU)       (1, None, 1024)           3938304   
_________________________________________________________________
dense_4 (Dense)              (1, None, 65)             66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


In [0]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a multinomial distribution to predict the word returned by the model
      predictions = predictions / temperature
      predicted_id = tf.multinomial(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted word as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [57]:
print(generate_text(model, start_string=u"wherefore art "))

wherefore art thou should here impossible me at your pleasure?
Ou shalt too well ? had thenced this wood,
And his heaven mourn but Lord Angelo,
And in thy redress shed to cross to shrink from hence,
But milk it well, and strike thee, thus must come about her: their liper
Servent for that. He shall have no more for Prating negligence:
If he were mine, I say amen! God pardon
Rame Edward, and go's good!
Some cried 'Chat dugger punish me or read!

NORTHUMBERLAND:
My lord, you talk. Now must I rat of blood and broad?

ROMEO:
Let me set at Corioli: thou'rt
Saint George, the time 'twixt your du'll march from me,
And cheer even in the coldest father's life;
Were but a timel ages note gone for his child
Snot braved me to a maid, and so took him, To make me misery.

GONZALO:
Merry, they would wish the hair
Be shall in Verona's swallow's wings:
Rescue, to make a stranger in the ingentious; nay, they kiss conspire
Yourself your lady mother in a male traitor:
I am the shore, to he from us,
Ir never

In [40]:
embedding_dim

256

In [0]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [0]:
optimizer = tf.train.AdamOptimizer()

In [37]:
# Training step
EPOCHS = 50

for epoch in range(EPOCHS):
    start = time.time()

    # initializing the hidden state at the start of every epoch
    # initially hidden is None
    hidden = model.reset_states()

    for (batch_n, (inp, target)) in enumerate(dataset):
          with tf.GradientTape() as tape:
              # feeding the hidden state back into the model
              # This is the interesting step
              predictions = model(inp)
              loss = tf.losses.sparse_softmax_cross_entropy(target, predictions)

          grads = tape.gradient(loss, model.trainable_variables)
          optimizer.apply_gradients(zip(grads, model.trainable_variables))

          if batch_n % 100 == 0:
              template = 'Epoch {} Batch {} Loss {:.4f}'
              print(template.format(epoch+1, batch_n, loss))

    # saving (checkpoint) the model every 5 epochs
    if (epoch + 1) % 5 == 0:
      model.save_weights(checkpoint_prefix.format(epoch=epoch))

    print ('Epoch {} Loss {:.4f}'.format(epoch+1, loss))
    print ('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 2.1002
Epoch 1 Batch 100 Loss 1.9132
Epoch 1 Loss 1.8017
Time taken for 1 epoch 12.58073878288269 sec

Epoch 2 Batch 0 Loss 1.7517
Epoch 2 Batch 100 Loss 1.6689
Epoch 2 Loss 1.6207
Time taken for 1 epoch 12.59295392036438 sec

Epoch 3 Batch 0 Loss 1.5582
Epoch 3 Batch 100 Loss 1.5316
Epoch 3 Loss 1.5160
Time taken for 1 epoch 12.75375771522522 sec

Epoch 4 Batch 0 Loss 1.4532
Epoch 4 Batch 100 Loss 1.4464
Epoch 4 Loss 1.4465
Time taken for 1 epoch 12.807005405426025 sec

Epoch 5 Batch 0 Loss 1.3867
Epoch 5 Batch 100 Loss 1.3859
Epoch 5 Loss 1.3912
Time taken for 1 epoch 12.69405746459961 sec

Epoch 6 Batch 0 Loss 1.3376
Epoch 6 Batch 100 Loss 1.3372
Epoch 6 Loss 1.3446
Time taken for 1 epoch 12.564063310623169 sec

Epoch 7 Batch 0 Loss 1.2967
Epoch 7 Batch 100 Loss 1.2919
Epoch 7 Loss 1.3020
Time taken for 1 epoch 12.555762529373169 sec

Epoch 8 Batch 0 Loss 1.2598
Epoch 8 Batch 100 Loss 1.2467
Epoch 8 Loss 1.2633
Time taken for 1 epoch 12.592432022094727 sec

Epoc

In [38]:
print(generate_text(model, start_string=u"HAMLET: "))

InvalidArgumentError: ignored

In [39]:
print(generate_text(model, start_string=u"ROMEO: "))

InvalidArgumentError: ignored

# Resources and Stretch Goals

## Stretch goals:
- Refine the training and generation of text to be able to ask for different genres/styles of Shakespearean text (e.g. plays versus sonnets)
- Train a classification model that takes text and returns which work of Shakespeare it is most likely to be from
- Make it more performant! Many possible routes here - lean on Keras, optimize the code, and/or use more resources (AWS, etc.)
- Revisit the news example from class, and improve it - use categories or tags to refine the model/generation, or train a news classifier
- Run on bigger, better data

## Resources:
- [The Unreasonable Effectiveness of Recurrent Neural Networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/) - a seminal writeup demonstrating a simple but effective character-level NLP RNN
- [Simple NumPy implementation of RNN](https://github.com/JY-Yoon/RNN-Implementation-using-NumPy/blob/master/RNN%20Implementation%20using%20NumPy.ipynb) - Python 3 version of the code from "Unreasonable Effectiveness"
- [TensorFlow RNN Tutorial](https://github.com/tensorflow/models/tree/master/tutorials/rnn) - code for training a RNN on the Penn Tree Bank language dataset
- [4 part tutorial on RNN](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) - relates RNN to the vanishing gradient problem, and provides example implementation
- [RNN training tips and tricks](https://github.com/karpathy/char-rnn#tips-and-tricks) - some rules of thumb for parameterizing and training your RNN